In [1]:
setwd("..")

In [2]:
source("Source/functions.R")
library("data.table")

In [3]:
snv_info <- fread("Data/snv_loci_v2.csv")
snvIDs <- snv_info[, SNVID]

chrmproperties <- fread("Report/artifact/chrmbases_properties.csv.gz")
setkey(chrmproperties, "pos")

In [4]:
MitoInfo <- fread(file = "Report/metadata/MitoInfo.csv")
dim(MitoInfo)

[1] 1717   19

In [5]:
CellInfo <- fread("Report/metadata/CellInfo.csv")
dim(CellInfo)

[1] 102   7

In [6]:
MouseInfo <- fread("Report/metadata/MouseInfo.csv")
dim(MouseInfo)

[1] 13  2

Load the original SNV table that had missing data at 9027 because of the 150/151 mixed cycles

In [7]:
basedifffreq_cutdemux_q30_unstranded <- fread("Report/SNVs/basedifffreq_cutdemux_sub500k_q30_unstranded.csv.gz")
basedifffreq_cutdemux_q30_unstranded[, ref := toupper(ref)]
basedifffreq_cutdemux_q30_unstranded[pos == 9027, list(LibraryMitoID, `=`, `A`, `C`, `G`, `T`)]

LibraryMitoID,=,A,C,G,T
<chr>,<int>,<int>,<int>,<int>,<int>
L1R24P1C_M1,1,28,0,0,0
L1R24P1C_M2,37,44,0,0,0
L1R24P1C_M3,53,46,0,0,0
L1R24P1C_M4,12,167,0,0,0
L1R24P1C_M5,0,1,0,0,0
L1R24P1C_M6,56,2,0,0,0
L1R24P1C_M7,1,38,0,0,0
L1R24P1C_M8,1,27,0,0,0
L1R24P1C_M9,10,3,0,0,0


Load 9027 matched and clipped counts

In [8]:
basedifffreq_end_q30_bysnv <- fread(file = "Report/artifact/basedifffreq_end_q30_bysnv.csv.gz")
setkey(basedifffreq_end_q30_bysnv, LibraryMitoID)

In [9]:
basedifffreq_9027_q30 <- basedifffreq_end_q30_bysnv[pos == 9027 & IsClip == "Y"][J(basedifffreq_cutdemux_q30_unstranded[pos == 9027, LibraryMitoID]), list(LibraryMitoID, `A`, `C`, `G`, `T`)]
basedifffreq_9027_q30[is.na(`A`), `A` := 0]
basedifffreq_9027_q30[is.na(`C`), `C` := 0]
basedifffreq_9027_q30[is.na(`G`), `G` := 0]
basedifffreq_9027_q30[is.na(`T`), `T` := 0]

In [10]:
basedifffreq_9027_q30

LibraryMitoID,A,C,G,T
<chr>,<int>,<int>,<int>,<int>
L1R24P1C_M1,12255,0,0,0
L1R24P1C_M2,255117,7,0,28
L1R24P1C_M3,290043,4,0,31
L1R24P1C_M4,127776,2,0,12
L1R24P1C_M5,110,0,0,0
L1R24P1C_M6,132,0,0,0
L1R24P1C_M7,5912,0,0,0
L1R24P1C_M8,101,0,0,0
L1R24P1C_M9,6099,1,0,2


Compensate the softclipped bases at 9027

In [11]:
basedifffreq_cutdemux_q30_unstranded[pos == 9027, `A` := `A` + basedifffreq_9027_q30[, `A`]]
basedifffreq_cutdemux_q30_unstranded[pos == 9027, `C` := `C` + basedifffreq_9027_q30[, `C`]]
## must be zero G
basedifffreq_cutdemux_q30_unstranded[pos == 9027, `=` := `=` + basedifffreq_9027_q30[, `G`]]
basedifffreq_cutdemux_q30_unstranded[pos == 9027, `T` := `T` + basedifffreq_9027_q30[, `T`]]
basedifffreq_cutdemux_q30_unstranded[pos == 9027, list(LibraryMitoID, `=`, `A`, `C`, `G`, `T`)]

LibraryMitoID,=,A,C,G,T
<chr>,<int>,<int>,<int>,<int>,<int>
L1R24P1C_M1,1,12283,0,0,0
L1R24P1C_M2,37,255161,7,0,28
L1R24P1C_M3,53,290089,4,0,31
L1R24P1C_M4,12,127943,2,0,12
L1R24P1C_M5,0,111,0,0,0
L1R24P1C_M6,56,134,0,0,0
L1R24P1C_M7,1,5950,0,0,0
L1R24P1C_M8,1,128,0,0,0
L1R24P1C_M9,10,6102,1,0,2


In [12]:
basedifffreq_cutdemux_q30_unstranded[pos == 9027, `depth` := `=` + `A` + `C` + `G` + `T` + `del`]
basedifffreq_cutdemux_q30_unstranded[pos == 9027, list(LibraryMitoID, depth, `=`, `A`, `C`, `G`, `T`, del)]

LibraryMitoID,depth,=,A,C,G,T,del
<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>
L1R24P1C_M1,12284,1,12283,0,0,0,0
L1R24P1C_M2,255233,37,255161,7,0,28,0
L1R24P1C_M3,290177,53,290089,4,0,31,0
L1R24P1C_M4,127969,12,127943,2,0,12,0
L1R24P1C_M5,111,0,111,0,0,0,0
L1R24P1C_M6,190,56,134,0,0,0,0
L1R24P1C_M7,5951,1,5950,0,0,0,0
L1R24P1C_M8,129,1,128,0,0,0,0
L1R24P1C_M9,6115,10,6102,1,0,2,0


In [13]:
head(basedifffreq_cutdemux_q30_unstranded)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,SNVID,pos,ref,depth,=,A,C,G,T,del
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9350,C,5431,5431,0,0,0,0,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9351,T,5572,5572,0,0,0,0,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9352,G,19115,19115,0,0,0,0,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9353,A,16857,16846,0,10,0,1,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9354,C,19213,19207,1,0,0,5,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9355,T,19324,19313,1,1,0,0,9


In [ ]:
fwrite(basedifffreq_cutdemux_q30_unstranded, file = "Report/SNVs/basedifffreq_cutdemux_sub500k_q30_unstranded_9027fixed.csv.gz")

In [15]:
basediffperc_cutdemux_q30_unstranded <- copy(basedifffreq_cutdemux_q30_unstranded)
basediffperc_cutdemux_q30_unstranded[, c("=", "A", "C", "G", "T", "del") := lapply(.SD, function(x) x / depth * 100),.SDcols = c("=", "A", "C", "G", "T", "del")]
dim(basediffperc_cutdemux_q30_unstranded)

[1] 3117733      29

In [16]:
depth_th <- 50
basediffperc_cutdemux_q30_unstranded_highdepth <- basediffperc_cutdemux_q30_unstranded[depth >= depth_th]
dim(basediffperc_cutdemux_q30_unstranded_highdepth)

[1] 1703153      29

In [17]:
head(basediffperc_cutdemux_q30_unstranded_highdepth)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,SNVID,pos,ref,depth,=,A,C,G,T,del
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9350,C,5431,100.00000,0.000000000,0.000000000,0,0.000000000,0.00000000
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9351,T,5572,100.00000,0.000000000,0.000000000,0,0.000000000,0.00000000
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9352,G,19115,100.00000,0.000000000,0.000000000,0,0.000000000,0.00000000
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9353,A,16857,99.93475,0.000000000,0.059322537,0,0.005932254,0.00000000
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9354,C,19213,99.96877,0.005204809,0.000000000,0,0.026024046,0.00000000
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9355,T,19324,99.94308,0.005174912,0.005174912,0,0.000000000,0.04657421


In [ ]:
fwrite(basediffperc_cutdemux_q30_unstranded_highdepth, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth.csv.gz")

SNV calls

In [19]:
vaf_th <- 0.05
basediffperc_cutdemux_q30_unstranded_highdepth_highaf <- basediffperc_cutdemux_q30_unstranded_highdepth[`A` >= vaf_th * 100 | `C` >= vaf_th * 100 | `G` >= vaf_th * 100 | `T` >= vaf_th * 100 | `del` >= vaf_th * 100]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf)

[1] 7967   29

In [20]:
head(basediffperc_cutdemux_q30_unstranded_highdepth_highaf)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,SNVID,pos,ref,depth,=,A,C,G,T,del
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9461,T,14335,0.048831531,0.04883153,99.902336938,0.000000000,0.00000000,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV3,9027,G,12284,0.008140671,99.99185933,0.000000000,0.000000000,0.00000000,0
L1R24P1C,730,L1R24P1C_M2,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B2--M2,⋯,SNV1,9461,T,75025,0.029323559,0.05064978,99.914695102,0.005331556,0.00000000,0
L1R24P1C,730,L1R24P1C_M2,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B2--M2,⋯,SNV3,9027,G,255233,0.014496558,99.97179048,0.002742592,0.000000000,0.01097037,0
L1R24P1C,730,L1R24P1C_M2,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B2--M2,⋯,SNV8,16125,A,57,87.719298246,0.00000000,5.263157895,7.017543860,0.00000000,0
L1R24P1C,730,L1R24P1C_M3,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B3--M3,⋯,SNV1,9461,T,86245,0.045220013,0.04869848,99.906081512,0.000000000,0.00000000,0


In [ ]:
fwrite(basediffperc_cutdemux_q30_unstranded_highdepth_highaf, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_highaf.csv.gz")

Take the overlapping SNV sites between BWA and STAR
Note, is_by_bwa is conditioned on the SNVs called by BWA at 5%, while `is_in_range` and `is_in_primer` are irrespective to SNV calls. 

In [22]:
pos_noctrl_starvsbwa <- fread("Report/bwa/snv_pos_noctrl.csv")
pos_noctrl_starvsbwa <- na.omit(pos_noctrl_starvsbwa[, intersect(all_star, all_bwa)])
pos_noctrl_starvsbwa <- data.table(pos = basediffperc_cutdemux_q30_unstranded_highdepth_highaf[IsCtrl == "N", sort(unique(pos))], is_by_bwa = ifelse(basediffperc_cutdemux_q30_unstranded_highdepth_highaf[IsCtrl == "N", sort(unique(pos))] %in% pos_noctrl_starvsbwa, "Y", "N"))
pos_noctrl_starvsbwa[, table(is_by_bwa)]

is_by_bwa
  N   Y 
 54 896 

In [23]:
pos_noctrl_qc <- merge.data.table(pos_noctrl_starvsbwa, chrmproperties[, c("pos", "is_in_range", "is_in_primer")], by = "pos")
pos_noctrl_qc[is_by_bwa == "Y" & is_in_range == "Y" & is_in_primer == "N", .N]

[1] 838

In [24]:
head(pos_noctrl_qc)

pos,is_by_bwa,is_in_range,is_in_primer
<int>,<chr>,<chr>,<chr>
1213,Y,Y,Y
1214,Y,Y,Y
1222,Y,Y,Y
1224,Y,Y,N
1225,Y,Y,N
1226,Y,Y,N


In [ ]:
fwrite(pos_noctrl_qc, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_highaf_qc.csv")

Filter for the BWA-STAR overlapping, in-range, out-of-primers SNV sites

In [26]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd <- basediffperc_cutdemux_q30_unstranded_highdepth_highaf[pos %in% pos_noctrl_qc[is_by_bwa == "Y" & is_in_range == "Y" & is_in_primer == "N", pos]]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd)

[1] 7157   29

In [27]:
head(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,SNVID,pos,ref,depth,=,A,C,G,T,del
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9461,T,14335,0.048831531,0.04883153,99.902336938,0.000000000,0.00000000,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV3,9027,G,12284,0.008140671,99.99185933,0.000000000,0.000000000,0.00000000,0
L1R24P1C,730,L1R24P1C_M2,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B2--M2,⋯,SNV1,9461,T,75025,0.029323559,0.05064978,99.914695102,0.005331556,0.00000000,0
L1R24P1C,730,L1R24P1C_M2,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B2--M2,⋯,SNV3,9027,G,255233,0.014496558,99.97179048,0.002742592,0.000000000,0.01097037,0
L1R24P1C,730,L1R24P1C_M2,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B2--M2,⋯,SNV8,16125,A,57,87.719298246,0.00000000,5.263157895,7.017543860,0.00000000,0
L1R24P1C,730,L1R24P1C_M3,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B3--M3,⋯,SNV1,9461,T,86245,0.045220013,0.04869848,99.906081512,0.000000000,0.00000000,0


In [ ]:
fwrite(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_highaf_qcfltd.csv.gz")

How many mitos/cells/mice support each SNV after QC filtering?

In [29]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl <- basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd[IsCtrl == "N"]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl)

[1] 6763   29

In [30]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[ref == "A", A := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[ref == "C", C := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[ref == "G", G := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[ref == "T", T := `=`]

In [31]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl <- melt.data.table(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl, measure.vars = c("A", "C", "G", "T", "del"), variable.name = "alt", value.name = "altperc")
setnames(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl, "=", "refperc")
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl)

[1] 33815    26

Keep only variant alleles

In [32]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl <- basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[ref != alt]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl)

[1] 27052    26

In [33]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[, summary(altperc)]
##    Min. 1st Qu.  Median    Mean 3rd Qu.    Max.

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   0.000   0.000  10.857   5.085 100.000 

Keep only variant alleles whose AF are >= 5% 

In [34]:
vaf_th <- 0.05
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl <- basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[altperc >= vaf_th * 100]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl)

[1] 6863   26

In [35]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[, alt := factor(alt, levels = c("A", "C", "G", "T", "del"))]
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[, mut := factor(paste0(ref, ">", alt), levels = paste0(rep(c("A", "C", "G", "T"), each = 5), ">", c("A", "C", "G", "T", "del")))]
setkey(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl, pos, alt)
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[, posmut := paste0(pos, ":", mut)]
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[, posmut := factor(posmut, levels = unique(posmut))]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl)

[1] 6863   28

In [36]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[, summary(altperc)]

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  5.000   7.317  15.855  42.322  96.926 100.000 

Get the number of sample support by variant alleles

In [37]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support <- basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_noctrl[, list(SNVID = unique(SNVID), nmice = uniqueN(MouseID), ncells = uniqueN(CellUID), nmitos = uniqueN(LibraryMitoID)), by = "posmut"]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support)

[1] 1032    5

In [38]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support[, pos := as.integer(sapply(strsplit(as.character(posmut), ":"), '[', 1))]
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support[, mut := sapply(strsplit(as.character(posmut), ":"), '[', 2)]
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support[, ref := sapply(strsplit(as.character(mut), ">"), '[', 1)]
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support[, alt := sapply(strsplit(as.character(mut), ">"), '[', 2)]
setcolorder(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support, c("posmut", "SNVID", "pos", "mut", "ref", "alt", "nmice", "ncells", "nmitos"))

In [39]:
head(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support)

posmut,SNVID,pos,mut,ref,alt,nmice,ncells,nmitos
<fct>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>
1224:A>G,SNV12,1224,A>G,A,G,3,5,5
1224:A>T,SNV12,1224,A>T,A,T,1,2,3
1225:G>A,SNV12,1225,G>A,G,A,2,2,2
1226:A>G,SNV12,1226,A>G,A,G,1,1,1
1228:G>A,SNV12,1228,G>A,G,A,3,4,6
1228:G>T,SNV12,1228,G>T,G,T,1,1,1


In [40]:
head(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support)

posmut,SNVID,pos,mut,ref,alt,nmice,ncells,nmitos
<fct>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>
1224:A>G,SNV12,1224,A>G,A,G,3,5,5
1224:A>T,SNV12,1224,A>T,A,T,1,2,3
1225:G>A,SNV12,1225,G>A,G,A,2,2,2
1226:A>G,SNV12,1226,A>G,A,G,1,1,1
1228:G>A,SNV12,1228,G>A,G,A,3,4,6
1228:G>T,SNV12,1228,G>T,G,T,1,1,1


In [ ]:
fwrite(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut.csv")

Get the number of sample support by SNV site

In [42]:
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support <- basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd[IsCtrl == "N", list(SNVID = unique(SNVID), nmice = uniqueN(MouseID), ncells = uniqueN(CellUID), nmitos = uniqueN(LibraryMitoID)), by = "pos"]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support)

[1] 838   5

In [43]:
head(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support)

pos,SNVID,nmice,ncells,nmitos
<int>,<chr>,<int>,<int>,<int>
9461,SNV1,13,100,1198
9027,SNV3,13,100,1308
16125,SNV8,13,62,187
16073,SNV8,7,7,9
16092,SNV8,11,29,52
16099,SNV8,12,44,174


In [ ]:
fwrite(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_highaf_qcfltd_support_bypos.csv")

Before filtering for nmitos/ncells/nmice >= k SNVs, first output them all

In [45]:
basediffperc_cutdemux_q30_unstranded_highdepth <- fread("Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth.csv.gz")
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut <- fread(file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut.csv")
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd <- basediffperc_cutdemux_q30_unstranded_highdepth[pos %in% basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut[, pos]]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd)

[1] 832872     29

In [46]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref == "A", A := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref == "C", C := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref == "G", G := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref == "T", T := `=`]

In [47]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd <- melt.data.table(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd, measure.vars = c("A", "C", "G", "T", "del"), variable.name = "alt", value.name = "altperc")
setnames(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd, "=", "refperc")
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd)

[1] 4164360      26

In [48]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd <- basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref != alt]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd)

[1] 3331488      26

In [49]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, alt := factor(alt, levels = c("A", "C", "G", "T", "del"))]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, mut := factor(paste0(ref, ">", alt), levels = paste0(rep(c("A", "C", "G", "T"), each = 5), ">", c("A", "C", "G", "T", "del")))]
setkey(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd, pos, alt)
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, posmut := paste0(pos, ":", mut)]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, posmut := factor(posmut, levels = unique(posmut))]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd)

[1] 3331488      28

In [50]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd <- basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[as.character(posmut) %in% as.character(basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut[, posmut])]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd)

[1] 1021830      28

In [51]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd <- basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, -(27:28)]
head(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,Enzyme,HasRCA,IsCtrl,SNVID,pos,ref,depth,refperc,alt,altperc
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<fct>,<dbl>
L1R24P1C,730,L1R24P1C_M2,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B2--M2,⋯,None,Y,N,SNV12,1224,A,196,100.00000,G,0.00000000
L1R24P1C,730,L1R24P1C_M4,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B4--M4,⋯,None,Y,N,SNV12,1224,A,1981,99.94952,G,0.05047956
L1R24P1C,730,L1R24P1C_M7,2020-09-04,3,Neuron,1,1,Mouse05_Neuron_1,B7--M7,⋯,None,Y,N,SNV12,1224,A,208,100.00000,G,0.00000000
L2R24P1C,730,L2R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,C1--M1,⋯,None,Y,N,SNV12,1224,A,164,100.00000,G,0.00000000
L2R24P1C,730,L2R24P1C_M2,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,C2--M2,⋯,None,Y,N,SNV12,1224,A,231,99.56710,G,0.43290043
L2R24P1C,730,L2R24P1C_M4,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,C4--M4,⋯,None,Y,N,SNV12,1224,A,2260,99.82301,G,0.17699115


In [ ]:
fwrite(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_qcfltd.csv.gz")

For all QC filtered SNVs, get the VAF table: mito by SNV

In [53]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd <- fread(file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_qcfltd.csv.gz")
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, alt := factor(alt, levels = c("A", "C", "G", "T", "del"))]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, mut := factor(paste0(ref, ">", alt), levels = paste0(rep(c("A", "C", "G", "T"), each = 5), ">", c("A", "C", "G", "T", "del")))]
setkey(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd, pos, alt)
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, posmut := paste0(pos, ":", mut)]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, posmut := factor(posmut, levels = unique(posmut))]
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd)

[1] 1021830      28

In [54]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_altperc_bymito_byposmut <- dcast.data.table(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd, LibraryMitoID ~ posmut, value.var = "altperc")
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_altperc_bymito_byposmut)

[1] 1702 1033

In [55]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_altperc_bymito_byposmut <- merge.data.table(MitoInfo, basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_altperc_bymito_byposmut, by = "LibraryMitoID", all.x = FALSE, all.y = TRUE)
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_altperc_bymito_byposmut)

[1] 1702 1051

In [56]:
head(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_altperc_bymito_byposmut)

LibraryMitoID,LibraryID,ExptID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,16124:C>G,16125:A>C,16125:A>G,16125:A>del,16133:T>A,16133:T>C,16133:T>G,16140:T>A,16140:T>C,16141:T>C
<chr>,<chr>,<int>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CTRR23P1B_M1,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(+)/(+) M1,⋯,0.00659098,2.761370,0.02953337,0.7051093,53.00940,0.5956113,8.996865,32.53553,0.1495886,0.05878895
CTRR23P1B_M10,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(-)/(-) M10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CTRR23P1B_M2,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(+)/(+) M2,⋯,0.00000000,3.478854,0.00000000,0.4092769,52.40642,0.0000000,9.625668,21.95122,0.6097561,0.00000000
CTRR23P1B_M3,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(-)/(+) M3,⋯,0.00000000,0.000000,0.00000000,0.0000000,0.00000,0.0000000,0.000000,0.00000,0.0000000,0.00000000
CTRR23P1B_M4,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(-)/(+) M4,⋯,0.00000000,1.639344,0.00000000,0.0000000,NA,NA,NA,NA,NA,NA
CTRR23P1B_M6,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(+)/(-) M6,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
fwrite(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_altperc_bymito_byposmut, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_qcfltd_altperc_bymito_byposmut.csv.gz")

For all QC filtered SNV sites, get the allele table: mitos by sites (A, AG, etc. whatever bases >= 5%)

In [58]:
basediffperc_cutdemux_q30_unstranded_highdepth <- fread("Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth.csv.gz")
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut <- fread(file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut.csv")
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd <- basediffperc_cutdemux_q30_unstranded_highdepth[pos %in% basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut[, pos]]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref == "A", A := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref == "C", C := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref == "G", G := `=`]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[ref == "T", T := `=`]

In [59]:
vaf_th <- 0.05
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, allele := apply(.SD, 1, function(x) paste0(c("A", "C", "G", "T", "-")[x >= vaf_th * 100], collapse = "")), .SDcols = c("A", "C", "G", "T", "del")]
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd[, table(allele)]

allele
     A     A-     AC    AC-    ACG   ACG-   ACGT    ACT     AG    AGT     AT 
298972     49    855     57     15      2      1      2   1439     25    505 
     C     C-     CG    CGT     CT      G     GT      T     T- 
204063     28    339      8   1015 100040    853 224601      3 

In [60]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_allele_bymito_bypos <- dcast.data.table(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd, LibraryMitoID ~ pos, value.var = "allele")
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_allele_bymito_bypos)

[1] 1702  839

In [61]:
basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_allele_bymito_bypos <- merge.data.table(MitoInfo, basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_allele_bymito_bypos, by = "LibraryMitoID", all.x = FALSE, all.y = TRUE)
dim(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_allele_bymito_bypos)

[1] 1702  857

In [62]:
head(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_allele_bymito_bypos)

LibraryMitoID,LibraryID,ExptID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,16118,16119,16120,16122,16123,16124,16125,16133,16140,16141
<chr>,<chr>,<int>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CTRR23P1B_M1,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(+)/(+) M1,⋯,AC,AC,AC,C,C,C,A,AGT,AT,T
CTRR23P1B_M10,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(-)/(-) M10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CTRR23P1B_M2,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(+)/(+) M2,⋯,AC,A,AC,C,C,C,A,AGT,AT,T
CTRR23P1B_M3,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(-)/(+) M3,⋯,A,A,A,C,C,C,A,T,T,T
CTRR23P1B_M4,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(-)/(+) M4,⋯,AC,A,A,C,NA,C,A,NA,NA,NA
CTRR23P1B_M6,CTRR23P1B,733,2020-08-28,3,,NA,NA,,(+)/(-) M6,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
fwrite(basediffperc_cutdemux_q30_unstranded_highdepth_qcfltd_allele_bymito_bypos, file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_qcfltd_allele_bymito_bypos.csv.gz")

For all QC filtered SNVs, get the depth table: mito by site  
Note, we need to recover the low-depth sites, so we have to go back to the original table before depth filtering. 

In [64]:
basedifffreq_cutdemux_q30_unstranded <- fread(file = "Report/SNVs/basedifffreq_cutdemux_sub500k_q30_unstranded_9027fixed.csv.gz")
basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut <- fread(file = "Report/SNVs/filter/basediffperc_cutdemux_sub500k_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut.csv")
basedifffreq_cutdemux_q30_unstranded_qcfltd <- basedifffreq_cutdemux_q30_unstranded[pos %in% basediffperc_cutdemux_q30_unstranded_highdepth_highaf_qcfltd_support_byposmut[, unique(pos)]]
basedifffreq_cutdemux_q30_unstranded_qcfltd_depth_bymito_bypos <- dcast.data.table(basedifffreq_cutdemux_q30_unstranded_qcfltd, LibraryMitoID ~ pos, value.var = "depth", fill = 0)
basedifffreq_cutdemux_q30_unstranded_qcfltd_depth_bymito_bypos <- MitoInfo[basedifffreq_cutdemux_q30_unstranded_qcfltd_depth_bymito_bypos, on = "LibraryMitoID"]

In [65]:
head(basedifffreq_cutdemux_q30_unstranded_qcfltd_depth_bymito_bypos)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,16118,16119,16120,16122,16123,16124,16125,16133,16140,16141
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CTRR23P1B,733,CTRR23P1B_M1,2020-08-28,3,,NA,NA,,(+)/(+) M1,⋯,14925,16873,11093,39717,46631,60689,27088,3190,2674,5103
CTRR23P1B,733,CTRR23P1B_M10,2020-08-28,3,,NA,NA,,(-)/(-) M10,⋯,1,2,2,1,1,2,2,3,3,0
CTRR23P1B,733,CTRR23P1B_M2,2020-08-28,3,,NA,NA,,(+)/(+) M2,⋯,800,873,583,2133,2584,3279,1466,187,164,285
CTRR23P1B,733,CTRR23P1B_M3,2020-08-28,3,,NA,NA,,(-)/(+) M3,⋯,146,132,127,96,103,136,142,101,141,121
CTRR23P1B,733,CTRR23P1B_M4,2020-08-28,3,,NA,NA,,(-)/(+) M4,⋯,56,52,52,59,49,68,61,37,47,44
CTRR23P1B,733,CTRR23P1B_M6,2020-08-28,3,,NA,NA,,(+)/(-) M6,⋯,2,3,2,0,1,2,2,2,1,2


In [ ]:
fwrite(basedifffreq_cutdemux_q30_unstranded_qcfltd_depth_bymito_bypos, file = "Report/SNVs/filter/basedifffreq_cutdemux_sub500k_q30_unstranded_qcfltd_depth_bymito_bypos.csv.gz")

Filter insertions

In [67]:
depth_th <- 50
vaf_th <- 0.05

mpileups_ins_cutdemux <- read.csv("Report/SNVs/ins/mpileups_ins_cutdemux_sub500k.csv.gz", as.is = TRUE, check.names = FALSE)
setDT(mpileups_ins_cutdemux)
mpileups_ins_cutdemux_highdepth_highaf <- mpileups_ins_cutdemux[depth >= depth_th & altperctot >= vaf_th * 100]
dim(mpileups_ins_cutdemux_highdepth_highaf)

[1] 134  39

In [68]:
head(mpileups_ins_cutdemux_highdepth_highaf)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,basefrac_bin10_T,is_in_range,is_in_primer,dist5,dist3,depth,alt,altfreq,altfreqtot,altperctot
<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<dbl>
L2R24P1C,730,L2R24P1C_M4,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,C4--M4,⋯,0.18181818,Y,N,71,117,96164,"AA,AAA,AC,AG,AT","4783,12,10,13,11",4829,5.021630
L4R24P1C,730,L4R24P1C_M7,2020-09-04,3,Neuron,1,4,Mouse05_Neuron_4,E7--M7,⋯,0.09090909,Y,Y,19,166,135,"AA,AACAC,ATC","1,1,5",7,5.185185
L5R24P1C,730,L5R24P1C_M4,2020-09-04,3,Neuron,1,4,Mouse05_Neuron_4,F4--M4,⋯,0.09090909,Y,Y,19,166,103,"AA,ATC","1,5",6,5.825243
L10R24P2C,730,L10R24P2C_M4,2020-09-04,3,Neuron,2,7,Mouse05_Neuron_7,A4--M4,⋯,0.00000000,Y,N,116,60,206,CA,14,14,6.796117
L16R24P3C,730,L16R24P3C_M7,2020-09-04,3,Neuron,3,12,Mouse05_Neuron_12,B7--M7,⋯,0.09090909,Y,Y,19,166,57,"ACGAAAC,ATC","2,1",3,5.263158
L11R23P1C,730,L11R23P1C_M1,2020-08-28,3,Neuron,1,2,Mouse04_Neuron_2,D1--M1,⋯,0.18181818,Y,N,121,80,233,AA,16,16,6.866953


In [ ]:
fwrite(mpileups_ins_cutdemux_highdepth_highaf, file = "Report/SNVs/filter/mpileups_ins_cutdemux_sub500k_highdepth_highaf.csv.gz", na = "NA")

In [70]:
chrmproperties <- fread("Report/artifact/chrmbases_properties.csv.gz")
mpileups_ins_cutdemux_highdepth_highaf_qcfltd <- mpileups_ins_cutdemux_highdepth_highaf[pos %in% chrmproperties[is_in_range == "Y" & is_in_primer == "N", pos]]
head(mpileups_ins_cutdemux_highdepth_highaf_qcfltd)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,basefrac_bin10_T,is_in_range,is_in_primer,dist5,dist3,depth,alt,altfreq,altfreqtot,altperctot
<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<dbl>
L2R24P1C,730,L2R24P1C_M4,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,C4--M4,⋯,0.18181818,Y,N,71,117,96164,"AA,AAA,AC,AG,AT","4783,12,10,13,11",4829,5.021630
L10R24P2C,730,L10R24P2C_M4,2020-09-04,3,Neuron,2,7,Mouse05_Neuron_7,A4--M4,⋯,0.00000000,Y,N,116,60,206,CA,14,14,6.796117
L11R23P1C,730,L11R23P1C_M1,2020-08-28,3,Neuron,1,2,Mouse04_Neuron_2,D1--M1,⋯,0.18181818,Y,N,121,80,233,AA,16,16,6.866953
L2R30P1B,732,L2R30P1B_M7,2021-02-12,3,Astrocyte,1,14,Mouse08_Astrocyte_14,B7--M7,⋯,0.00000000,Y,N,85,91,1255,"CA,CACAA,CACAAC,CACAACCAACAT","50,21,13,1",85,6.772908
L4R30P1B,732,L4R30P1B_M4,2021-02-12,3,Astrocyte,1,15,Mouse08_Astrocyte_15,D4--M4,⋯,0.18181818,Y,N,71,117,9283,"AA,AAA,AC,AT","1502,3,4,1",1510,16.266293
L4R30P1B,732,L4R30P1B_M9,2021-02-12,3,Astrocyte,1,15,Mouse08_Astrocyte_15,D9--M9,⋯,0.09090909,Y,N,88,100,81,CC,6,6,7.407407


In [ ]:
fwrite(mpileups_ins_cutdemux_highdepth_highaf_qcfltd, file = "Report/SNVs/filter/mpileups_ins_cutdemux_sub500k_highdepth_highaf_qcfltd.csv.gz")

In [72]:
sessionInfo()

R version 4.4.0 (2024-04-24)
Platform: x86_64-pc-linux-gnu
Running under: openSUSE Tumbleweed

Matrix products: default
BLAS:   /home/lenovo/Applications/R/4.4.0/lib64/R/lib/libRblas.so 
LAPACK: /home/lenovo/Applications/R/4.4.0/lib64/R/lib/libRlapack.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: America/New_York
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] data.table_1.16.2

loaded via a namespace (and not attached):
 [1] digest_0.6.37     IRdisplay_1.1     utf8_1.2.4        base64enc_0.1-3  
 [5] fastmap_1.2.0     glue_1.8